In [ ]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import openai
from openai import OpenAI
from sklearn.metrics import accuracy_score, classification_report
import json

## 1st Prompt with Traditional Chinese example and English definition

In [ ]:
# Set your OpenAI API key
openai.api_key ='your api key'


# Load the dataset
df = pd.read_csv('https://raw.githubusercontent.com/Letieateat/Sentiment-Analysis-on-Traditional-Chinese-Lyrics/main/dataset/final_sentiments.csv')

def classify_emotion(lyrics):
    prompt_text = f"""
    You are trained to analyze and detect the sentiment of given lyrics and return the results in JSON format.
    Example of Expected JSON Response Format:

    sentiment_all: Tag from Task 1, sentiment_4: Tag from Task 2, sentiment_2: Tag from Task 3

    Please detect the sentiment based on the following definition and examples:

    Anxious: worried and nervous; eager to do something; wanting very much for something to happen
    e.g. 該怎麽走 誰來告訴我

    Angry: having a strong feeling against someone who has behaved badly, making you want to shout at them or hurt them
    e.g. 你煩不煩 總考驗我多勇敢

    Excited: feeling very happy and enthusiastic
    e.g. 點起熱情的燭光 享受玫瑰的芳香

    Happy: feeling, showing, or causing pleasure or satisfaction; full of enjoyment and pleasure
    e.g. 一想到你呀 就讓我快樂

    Relaxed: feeling happy and comfortable because nothing is worrying you; comfortable and informal
    e.g. 人生很長 來得及任何事

    Serene: peaceful and calm; worried by nothing
    e.g. 讓我釋放 然後慢慢寬廣

    Sad: unhappy or sorry; not satisfactory or pleasant
    e.g. 我的淚光 承載不了

    Negative: Anxious; Angry; Sad

    Positive:  Excited; Happy; Serene; Relaxed

    You'll be provided a dataset containing Traditional Chinese lyrics of different songs.
    You need to complete 3 tasks. Select ONLY ONE TAG from the provided options for each task.
    (Task 1 : sentiment_all) Tags:
    - Anxious
    - Angry
    - Excited
    - Happy
    - Relaxed
    - Serene
    - Sad

    (Task 2 : sentiment_4) Tags:
    - Angry/Anxious
    - Excited/Happy
    - Serene/Relaxed
    - Sad

    (Task 3 : sentiment_2) Tags:
    - Negative
    - Positive

    Lyrics: {lyrics}
    """
    # Call the OpenAI API
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {
                    "role": "system",
                    "content": prompt_text
                },
                {
                    "role": "user",
                    "content": lyrics
                }
            ]
        )
        if response.choices:
            content = response.choices[0].message.content
            data = json.loads(content)
            # Normalize the data if it's not in the expected format
            sentiment_all = normalize_sentiment(data.get("sentiment_all", "Unknown"))
            sentiment_4 = normalize_sentiment(data.get("sentiment_4", "Unknown"))
            sentiment_2 = normalize_sentiment(data.get("sentiment_2", "Unknown"))
            return {
                "gpt_sentiment_all": data.get("sentiment_all", "Unknown"),
                "gpt_sentiment_4": data.get("sentiment_4", "Unknown"),
                "gpt_sentiment_2": data.get("sentiment_2", "Unknown"),
                "original_response": content
            }
        else:
            return {"gpt_sentiment_all": None, "gpt_sentiment_4": None, "gpt_sentiment_2": None}
    except Exception as e:
        print(f"Error handling response: {str(e)}")
        return {"gpt_sentiment_all": None, "gpt_sentiment_4": None, "gpt_sentiment_2": None}

def normalize_sentiment(sentiment):
    if isinstance(sentiment, list):
        # Convert list to string if needed, or choose a handling strategy
        return ', '.join(sentiment)
    return sentiment

# Apply to the dataset
results = df['untokenized_lyrics'].apply(classify_emotion)

# add results to dataframe
results_df = pd.DataFrame(results.tolist())
df = pd.concat([df, results_df], axis=1)

# Save the predictions to a new CSV file
df.to_csv('gpt3.5_sentiments.csv', index=False)

Error handling response: Expecting value: line 1 column 1 (char 0)
Error handling response: Expecting value: line 1 column 1 (char 0)
  Track Name Artist Name                                       Artist Genre  \
0    1234567  Cheer Chen                 mandopop, taiwan singer-songwriter   
1     100種生活    Crowd Lu   mandopop, taiwan indie, taiwan singer-songwriter   
2      612星球       S.H.E  c-pop girl group, mandopop, taiwan idol pop, t...   
3   80%完美的日子  Cheer Chen                 mandopop, taiwan singer-songwriter   
4         一一   Hebe Tien                                           mandopop   

   Popularity  Acousticness  Danceability  Duration (ms)  Energy  \
0          29         0.151         0.651         216560   0.659   
1          44         0.650         0.721         287427   0.400   
2          28         0.654         0.475         301347   0.432   
3          28         0.632         0.454         278147   0.422   
4          33         0.492         0.423         2

In [ ]:
# Load the CSV file into a DataFrame
file_path = 'https://raw.githubusercontent.com/Letieateat/Sentiment-Analysis-on-Traditional-Chinese-Lyrics/main/dataset/gpt3.5_sentiments.csv'
df = pd.read_csv(file_path)

In [ ]:
# define a function to convert data into string
def standardize_sentiments(sentiments):
    if isinstance(sentiments, list):
        return ', '.join(sentiments)
    return sentiments

# apply the function
df['gpt_sentiment_all'] = df['gpt_sentiment_all'].apply(standardize_sentiments)
df['gpt_sentiment_4'] = df['gpt_sentiment_4'].apply(standardize_sentiments)
df['gpt_sentiment_2'] = df['gpt_sentiment_2'].apply(standardize_sentiments)

df.dropna(subset=['human_all', 'gpt_sentiment_all'], inplace=True)
df.dropna(subset=['human_4', 'gpt_sentiment_4'], inplace=True)
df.dropna(subset=['human_2', 'gpt_sentiment_2'], inplace=True)

# Define the expected sentiment categories
expected_all = ['Angry', 'Anxious', 'Excited', 'Happy', 'Relaxed', 'Sad', 'Serene', 'Grateful', 'Spooky']
expected_4 = ['Angry/Anxious', 'Excited/Happy', 'Serene/Relaxed',  'Sad']
expected_2 = ['Negative', 'Positive']

# Filter the data to include only rows with expected categories
df = df[df['gpt_sentiment_all'].isin(expected_all)]
df = df[df['gpt_sentiment_4'].isin(expected_4)]
df = df[df['gpt_sentiment_2'].isin(expected_2)]


# Calculate accuracy
accuracy_all = accuracy_score(df['human_all'], df['gpt_sentiment_all'])
accuracy_4 = accuracy_score(df['human_4'], df['gpt_sentiment_4'])
accuracy_2 = accuracy_score(df['human_2'], df['gpt_sentiment_2'])

print(f"Accuracy for All Sentiments: {accuracy_all}")
print(f"Accuracy for 4 Sentiments: {accuracy_4}")
print(f"Accuracy for 2 Sentiments: {accuracy_2}")

# Classification Report
print("\nClassification Report for All Sentiments:")
print(classification_report(df['human_all'], df['gpt_sentiment_all'], zero_division=0))

print("\nClassification Report for 4 Sentiments:")
print(classification_report(df['human_4'], df['gpt_sentiment_4'], zero_division=0))

print("\nClassification Report for 2 Sentiments:")
print(classification_report(df['human_2'], df['gpt_sentiment_2'], zero_division=0))

Accuracy for All Sentiments: 0.5666666666666667
Accuracy for 4 Sentiments: 0.5545454545454546
Accuracy for 2 Sentiments: 0.7878787878787878

Classification Report for All Sentiments:
              precision    recall  f1-score   support

       Angry       0.78      0.36      0.49        59
     Anxious       0.61      0.17      0.26       138
     Excited       0.40      0.71      0.51        62
    Grateful       0.00      0.00      0.00         0
       Happy       0.46      0.68      0.55       111
     Relaxed       0.67      0.05      0.09       125
         Sad       0.67      0.89      0.77       403
      Serene       0.32      0.38      0.34        92
      Spooky       0.00      0.00      0.00         0

    accuracy                           0.57       990
   macro avg       0.43      0.36      0.33       990
weighted avg       0.60      0.57      0.51       990


Classification Report for 4 Sentiments:
                precision    recall  f1-score   support

 Angry/Anxious

In [ ]:
incorrect_predictions_all = []
incorrect_predictions_4 = []
incorrect_predictions_2 = []


for index, row in df.iterrows():
    if row['human_all'] != row['gpt_sentiment_all']:
        incorrect_predictions_all.append(row)
    if row['human_4'] != row['gpt_sentiment_4']:
        incorrect_predictions_4.append(row)
    if row['human_2'] != row['gpt_sentiment_2']:
        incorrect_predictions_2.append(row)

df_incorrect_all = pd.DataFrame(incorrect_predictions_all)
df_incorrect_4 = pd.DataFrame(incorrect_predictions_4)
df_incorrect_2 = pd.DataFrame(incorrect_predictions_2)


df_incorrect_all.to_csv('gpt3.5_errors_all.csv', index=False)
df_incorrect_4.to_csv('gpt3.5_errors_4.csv', index=False)
df_incorrect_2.to_csv('gpt3.5_errors_2.csv', index=False)

print(f"Saved {len(df_incorrect_all)} incorrect predictions for all sentiments to 'gpt3.5_errors_all.csv'")
print(f"Saved {len(df_incorrect_4)} incorrect predictions for 4 sentiments to 'gpt3.5_errors_4.csv'")
print(f"Saved {len(df_incorrect_2)} incorrect predictions for 2 sentiments to 'gpt3.5_errors_2.csv'")


Saved 429 incorrect predictions for all sentiments to 'gpt3.5_errors_all.csv'
Saved 441 incorrect predictions for 4 sentiments to 'gpt3.5_errors_4.csv'
Saved 210 incorrect predictions for 2 sentiments to 'gpt3.5_errors_2.csv'
